In [10]:
import pandas as pd
import numpy as np
import datetime
import scipy.stats as sps

In [ ]:
mydata = pd.read_csv('card transactions_filled.csv')
mydata['Date'] = pd.to_datetime(mydata['Date'],format='%Y/%m/%d')

# Variable Creation

## Amount & Frequency Variables

To establish the normal amount and frequency level of an entity (card holder, merchant), calculate summary statistics for the entity's transaction in the various time windows (current day, 1 day, 3 days, 7 days, 14 days, 30 days) for each transaction record. 

In [ ]:
def genAmountVariables(df,col,days):
    df = df[[*col,'Date','Amount','Recnum']]
    d = str(days+1)+'d'
    byColDate = df.groupby([*col]).rolling(d, on="Date").agg({'Amount':'mean','Recnum':'max'})
    byColDate.columns = ['Average','Recnum']
    byColDate = byColDate.reset_index(drop=True).set_index('Recnum')
    byColDate['Max'] = df.groupby([*col]).rolling(d, on="Date")\
        .agg({'Amount':'max','Recnum':'max'}).reset_index().set_index('Recnum').Amount
    byColDate['Median'] = df.groupby([*col]).rolling(d, on="Date")\
        .agg({'Amount':'median','Recnum':'max'}).reset_index().set_index('Recnum').Amount
    byColDate['Total'] = df.groupby([*col]).rolling(d, on="Date")\
        .agg({'Amount':'sum','Recnum':'max'}).reset_index().set_index('Recnum').Amount
    byColDate['Count'] = df.groupby([*col]).rolling(d, on="Date")\
        .agg({'Amount':'count','Recnum':'max'}).reset_index().set_index('Recnum').Amount
    return byColDate

In [ ]:
%%time
colList = [['Cardnum'],['Merchnum'],['Cardnum','Merchnum'],['Cardnum','Merch zip'],['Cardnum','Merch state']]
daysList = [0,1,3,7,14,30]
test = mydata
variables = pd.DataFrame(columns=['Recnum','Date','Amount','Average','Max','Median','Total','Count'])
variables[['Recnum','Date','Amount']] = test[['Recnum','Date','Amount']]
variables = variables.set_index('Recnum')
for col in colList:
    colName = ''
    for i in col:
        colName = colName + '_' + i
    for days in daysList:
        result = genAmountVariables(test,col,days)
        rsuffix = '_'+colName+'_'+str(days)
        variables = variables.join(result,rsuffix=rsuffix)
variables = variables.drop(columns=['Average','Max','Median','Total','Count'])

In [ ]:
%%time
colIndex = []
for i in range(2,152):
    if (i-1)%5 != 0:
        colIndex.append(i)
cols_to_divid = variables.columns[colIndex]
for col in cols_to_divid:
    new_col_name = 'Actual/'+col
    variables[new_col_name] = variables['Amount']/variables[col]

## Recency Variables

For each transaction record, calculate the time lapse between the last transaction by the same card at the same merchant, or same zip code, same State. 

In [ ]:
%%time
df = mydata.set_index('Recnum')
variables = variables.assign(Date_Card = df.groupby(["Cardnum"])['Date'].diff().fillna(0).dt.days,
               Date_Merch = df.groupby(["Merchnum"])['Date'].diff().fillna(0).dt.days,
               Date_Card_Merch = df.groupby(["Cardnum","Merchnum"])['Date'].diff().fillna(0).dt.days,
               Date_Card_Zip = df.groupby(["Cardnum","Merch zip"])['Date'].diff().fillna(0).dt.days,
               Date_Card_State = df.groupby(["Cardnum","Merch state"])['Date'].diff().fillna(0).dt.days)

## Velocity Change Variables

In [ ]:
topColList = ['Count__Cardnum_0','Count__Cardnum_1','Count__Merchnum_0','Count__Merchnum_1','Total__Cardnum_0','Total__Cardnum_1','Total__Merchnum_0','Total__Merchnum_1']
downListday = [7, 14, 30]
for col in topColList:
    for i in downListday:
        for downname in ['Count__Cardnum_','Total__Cardnum_','Count__Merchnum_','Total__Merchnum_']:
            name = downname+str(i)
            varname = col+'/'+name
            variables[varname] = variables[col]/(variables[name]/(i+1))

In [ ]:
variables['Fraud']=mydata.set_index('Recnum')['Fraud']

In [ ]:
variables.to_csv('variables.csv')

# Feature Selection

In [30]:
variables = pd.read_csv('variables.csv')
variables = variables.drop(columns = ['Unnamed: 0'])
variables['Date'] = pd.to_datetime(variables['Date'],format='%Y/%m/%d')
variables.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,Total__Merchnum_1Count__Merchnum_7,Total__Merchnum_1Total__Merchnum_7,Total__Merchnum_1Count__Cardnum_14,Total__Merchnum_1Total__Cardnum_14,Total__Merchnum_1Count__Merchnum_14,Total__Merchnum_1Total__Merchnum_14,Total__Merchnum_1Count__Cardnum_30,Total__Merchnum_1Total__Cardnum_30,Total__Merchnum_1Count__Merchnum_30,Total__Merchnum_1Total__Merchnum_30
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,143.840000,4.417012,539.40,149.005525,269.700000,8.281898,1114.76,307.944751,557.380000,17.115922
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0,...,6874.708571,8.000000,90230.55,2871.755251,12890.078571,15.000000,186476.47,5934.960853,26639.495714,31.000000
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,793.209843,8.000000,568136.55,3183.016135,1434.688258,14.770570,1174148.87,6578.233346,2877.815858,29.562589
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,858.248499,7.094196,696790.50,192483.563536,1451.646875,12.479536,1440033.70,397799.364641,2834.712008,24.368456
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,89.920000,8.000000,168.60,46.574586,168.600000,15.000000,348.44,96.254144,348.440000,31.000000


### Seperate out of time data

In [31]:
v_oot = variables[variables['Date']>'2010-10-31']

## all other records
v_bfoot = variables.drop(v_oot.index)

## Selected features using KS&FDR

Use KS score to measure the similarity of variable values in fraud and non-fraud transactions. 
Use FDR to measure the ability of a variable to capture fraudulent transactions within top or bottom 3% of its values. 

In [77]:
variables = pd.read_csv('variables.csv')
variables['Date'] = pd.to_datetime(variables['Date'],format='%Y/%m/%d')
variables = variables.set_index('Recnum')
variables = variables.drop(columns = ['Merchnum','Merch description','Merch state','Merch zip','Transtype'])

In [78]:
# Separate Training and Testing data 
cutoff = datetime.datetime.strptime('2010/11/1','%Y/%m/%d')
train = variables[variables['Date']<cutoff]

In [79]:
# Introduce random variable to validate KS and FDR calculation
train['Random'] = np.array(np.random.ranf(len(train)))
train = train.drop(columns='Date')

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [80]:
# calculate FDR score for a variable
# input format: df = pd.DataFrame([Variable,Label])
def getFDR(df, percentage):
    df.columns = ['Variable','Label']
    num_rows = round(len(df)*percentage/100)
    num_bads = sum(df['Label'])
    asc = df.sort_values('Variable',ascending=True).head(num_rows)
    desc = df.sort_values('Variable',ascending=False).head(num_rows)
    FDR1 = sum(asc.iloc[:,1])/num_bads
    FDR2 = sum(desc.iloc[:,1])/num_bads
    FDR = max(FDR1,FDR2)
    return FDR

In [81]:
%%time
goods = train[train['Fraud']==0]
bads = train[train['Fraud']==1]
KS_FDR = []
for i in range(2,375):
    ks = sps.ks_2samp(goods.iloc[:,i],bads.iloc[:,i])
    FDR = getFDR(train.iloc[:,[i,373]],3)
    KS_FDR.append([train.columns[i],ks[0],FDR])
KS_FDR = pd.DataFrame(KS_FDR,columns=['Variable','KS','FDR'])

CPU times: user 11.2 s, sys: 197 ms, total: 11.4 s
Wall time: 11.4 s


In [82]:
KS_FDR.sort_values('KS',ascending=False).head(10)

,Variable,KS,FDR
1,Fraud,1.000000,0.029556
80,Total__Cardnum_Merch zip_7,0.648851,0.030837
50,Total__Cardnum_Merchnum_7,0.647587,0.031182
105,Total__Cardnum_Merch state_3,0.645691,0.031907
115,Total__Cardnum_Merch state_14,0.644039,0.030757
55,Total__Cardnum_Merchnum_14,0.642068,0.031650
75,Total__Cardnum_Merch zip_3,0.640971,0.032719
110,Total__Cardnum_Merch state_7,0.640381,0.030942
45,Total__Cardnum_Merchnum_3,0.639911,0.032767
100,Total__Cardnum_Merch state_1,0.639564,0.030927


In [83]:
## remove unwanted variables created for inspection: Fraud, Random
v_selected = KS_FDR[KS_FDR['Variable']!='Fraud']
v_selected = v_selected[v_selected['Variable']!='Random']

In [84]:
# Total number of variables
len(v_selected)

372

In [85]:
v_selected180 = v_selected[0:180]
v_180 = list(v_selected180['Variable'])
v_filter = v_180+['Fraud','Recnum']

In [86]:
v_oot_sltd = v_oot.loc[:,v_oot.columns.isin(v_filter)]
v_bfoot_sltd = v_bfoot.loc[:,v_bfoot.columns.isin(v_filter)]

## wrapper selection

In [87]:
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [95]:
from sklearn.linear_model import LogisticRegression

In [106]:
lr = LogisticRegression()
sfs2 = SFS(lr, 
           k_features=15, 
           forward=True, 
           floating=False, 
           scoring='accuracy',
           cv=0)
X = v_bfoot_sltd.drop(columns = ['Recnum','Amount','Fraud'])
y = v_bfoot_sltd['Fraud']

In [ ]:
%%time
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

sfs2 = sfs2.fit(X, y)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number o

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number o

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number o

In [104]:
v_list2 = sfs2.k_feature_names_

In [105]:
v_list2

('Total__Cardnum_1',
 'Count__Cardnum_3',
 'Total__Cardnum_30',
 'Median__Cardnum_Merchnum_14',
 'Total__Cardnum_Merchnum_14',
 'Count__Cardnum_Merch zip_1',
 'Max__Cardnum_Merch zip_7',
 'Count__Cardnum_Merch zip_30',
 'Count__Cardnum_Merch state_1',
 'Total__Cardnum_Merch state_30',
 'Actual/Max__Cardnum_1',
 'Actual/Total__Cardnum_1',
 'Actual/Total__Cardnum_3',
 'Actual/Max__Cardnum_7',
 'Actual/Median__Cardnum_30')